In [36]:
from util import load_dataset
import numpy as np

In [42]:
train, labels = load_dataset('./data/train.csv')

In [20]:
from keras.layers import Dense, Convolution2D, MaxPooling2D, AveragePooling2D, Input, Concatenate, Flatten, Dropout
from keras.models import Model

In [44]:
train = train[:,:,:,np.newaxis]
train.shape

(42000, 28, 28, 1)

In [88]:
def get_model():
    inp = Input(shape=(28,28,1))
    
    x_1 = Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(inp)
    x_1 = Convolution2D(filters=64, kernel_size=(5,5), activation='relu', padding='same')(x_1)
    x_1 = AveragePooling2D(pool_size=(2, 2))(x_1)
    x_1 = Convolution2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(x_1)
    x_1 = MaxPooling2D(pool_size=(3, 3))(x_1)
    x_1 = Dropout(0.3)(x_1)
    x_1 = Flatten()(x_1)
    
    
    x_2 = Convolution2D(filters=32, kernel_size=(3, 3),activation='relu')(inp)
    x_2 = Convolution2D(filters=64, kernel_size=(3, 3), activation='relu')(x_2)
    x_2 = MaxPooling2D(pool_size=(2, 2))(x_2)
    x_2=Dropout(0.4)(x_2)
    x_2=Flatten()(x_2)
    
    x = Concatenate()([x_1, x_2])
    
    x=Dense(200, activation='relu')(x)
    x=Dropout(0.5)(x)
    
    out = Dense(10, activation='softmax')(x)
    model = Model(inputs = inp, outputs = out)
    
    model.summary()
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
    return model

In [89]:
model = get_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 28, 28, 32)   320         input_14[0][0]                   
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 28, 28, 64)   51264       conv2d_44[0][0]                  
__________________________________________________________________________________________________
average_pooling2d_11 (AveragePo (None, 14, 14, 64)   0           conv2d_45[0][0]                  
__________________________________________________________________________________________________
conv2d_47 

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
train_x, val_x, train_y, val_y = train_test_split(train, labels, stratify=labels, train_size=0.9, random_state=12345)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [92]:
model.fit(train_x, train_y,
                    validation_data=(val_x, val_y),
                    epochs=20)

Train on 37800 samples, validate on 4200 samples
Epoch 1/20
37800/37800 [==============================] - 14s 359us/step - loss: 0.1813 - acc: 0.9440 - val_loss: 0.0918 - val_acc: 0.9733
Epoch 2/20
37800/37800 [==============================] - 13s 340us/step - loss: 0.0709 - acc: 0.9789 - val_loss: 0.0440 - val_acc: 0.9860
Epoch 3/20
37800/37800 [==============================] - 13s 343us/step - loss: 0.0506 - acc: 0.9843 - val_loss: 0.0478 - val_acc: 0.9848
Epoch 4/20
37800/37800 [==============================] - 13s 340us/step - loss: 0.0431 - acc: 0.9869 - val_loss: 0.0470 - val_acc: 0.9852
Epoch 5/20
37800/37800 [==============================] - 13s 340us/step - loss: 0.0365 - acc: 0.9888 - val_loss: 0.0469 - val_acc: 0.9874
Epoch 6/20
37800/37800 [==============================] - 13s 341us/step - loss: 0.0316 - acc: 0.9904 - val_loss: 0.0378 - val_acc: 0.9879
Epoch 7/20
37800/37800 [==============================] - 13s 343us/step - loss: 0.0281 - acc: 0.9908 - val_loss: 0.0

In [64]:
import pandas as pd

In [93]:
test_pd = pd.read_csv('./data/test.csv')

test = test_pd.values.reshape((test_pd.shape[0],28,28,1))

In [94]:
test.shape

(28000, 28, 28, 1)

In [95]:
preds = model.predict(test)

In [96]:
preds_val = np.argmax(preds, axis=1)

In [97]:
subm = pd.read_csv('./data/sample_submission.csv')
subm['Label']=preds_val
subm.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [98]:
subm.to_csv('submissions/conv.csv', index=False)